[//]: # ('cell_type':'markdown', "metadata":{'slideshow': {'slide_type': 'slide'}})
## Introduction to visualizing data in the eeghdf files

In [1]:
# %load explore-eeghdf-files-basics.py
# Here is an example of how to do basic exploration of what is in the eeghdf file. I show how to discover the fields in the file and to plot them.
# 
# I have copied the stacklineplot from my python-edf/examples code to help with display. Maybe I will put this as a helper or put it out as a utility package to make it easier to install.

from __future__ import print_function, division, unicode_literals
import pandas as pd
import numpy as np
import h5py
from pprint import pprint

import ipywidgets
from IPython.display import display

import eegvis.stacklineplot
import eegvis.stackplot_bokeh as stacklineplot
from bokeh.io import output_notebook, push_notebook
import bokeh.plotting as bplt
from bokeh.plotting import show
output_notebook()

Loading BokehJS ...

In [2]:
hdf = h5py.File('./archive/YA2741BS_1-1+.eeghdf') # 5mo boy 

In [3]:
pprint(list(hdf.items()))
pprint(list(hdf['patient'].attrs.items()))

[('patient', <HDF5 group "/patient" (0 members)>),
 ('record-0', <HDF5 group "/record-0" (10 members)>)]
[('patient_name', '906, Subject'),
 ('patientcode', '0bf1acf3d68c50938e6f9dd40906f3ad'),
 ('gender', 'Male'),
 ('birthdate', '1990-01-01'),
 ('patient_additional', ''),
 ('gestatational_age_at_birth_days', -1.0),
 ('born_premature', 'unknown')]


In [4]:
rec = hdf['record-0']
pprint(list(rec.items()))
pprint(list(rec.attrs.items()))
years_old = rec.attrs['patient_age_days']/365
pprint("age in years: %s" % years_old)

[('edf_annotations', <HDF5 group "/record-0/edf_annotations" (3 members)>),
 ('physical_dimensions',
  <HDF5 dataset "physical_dimensions": shape (36,), type "|O">),
 ('prefilters', <HDF5 dataset "prefilters": shape (36,), type "|O">),
 ('signal_digital_maxs',
  <HDF5 dataset "signal_digital_maxs": shape (36,), type "<i4">),
 ('signal_digital_mins',
  <HDF5 dataset "signal_digital_mins": shape (36,), type "<i4">),
 ('signal_labels', <HDF5 dataset "signal_labels": shape (36,), type "|O">),
 ('signal_physical_maxs',
  <HDF5 dataset "signal_physical_maxs": shape (36,), type "<f8">),
 ('signal_physical_mins',
  <HDF5 dataset "signal_physical_mins": shape (36,), type "<f8">),
 ('signals', <HDF5 dataset "signals": shape (36, 446000), type "<i2">),
 ('transducers', <HDF5 dataset "transducers": shape (36,), type "|O">)]
[('start_isodatetime', '1990-06-17 13:20:56'),
 ('end_isodatetime', '1990-06-17 13:58:06'),
 ('number_channels', 36),
 ('number_samples_per_channel', 446000),
 ('sample_frequen

In [5]:
signals = rec['signals']
labels = rec['signal_labels']
electrode_labels = [str(s,'ascii') for s in labels]
numbered_electrode_labels = ["%d:%s" % (ii, str(labels[ii], 'ascii')) for ii in range(len(labels))]
print(signals.shape)

(36, 446000)


[//]: # ('cell_type':'markdown', "metadata":{'slideshow': {'slide_type': 'slide'}})
#### Simple visualization of EEG (electrodecrement seizure pattern)

In [6]:
# search identified spasms at 1836, 1871, 1901, 1939
fig=stacklineplot.show_epoch_centered(signals, 1836,
                        epoch_width_sec=15,
                        chstart=0, chstop=19, fs=rec.attrs['sample_frequency'],
                        ylabels=electrode_labels, yscale=3.0)
show(fig)


In [7]:
annot = rec['edf_annotations']
#print(list(annot.items()))
#annot['texts'][:]

In [8]:
signals.shape

(36, 446000)

In [9]:
antext = [s.decode('utf-8') for s in annot['texts'][:]]
starts100ns = [xx for xx in annot['starts_100ns'][:]]
len(starts100ns), len(antext)

(66, 66)

In [10]:
df = pd.DataFrame(data=antext, columns=['text'])
df['starts100ns'] = starts100ns
df['starts_sec'] = df['starts100ns']/10**7

In [11]:
df # look at the annotations

,text,starts100ns,starts_sec
0,REC START Biocal CAL,0,0.000
1,A1+A2 OFF,11400000,1.140
2,PAT Biocal EEG,61900000,6.190
3,PAT A_DB EEG,114200000,11.420
4,IMP CHECK ON,564500000,56.450
5,IMP CHECK OFF,593500000,59.350
6,resting on right sid,731150000,73.115
7,head on right side,772180000,77.218
8,head on neckroll,809810000,80.981
9,nursnig quietly,856350000,85.635


In [12]:
df[df.text.str.contains('sz',case=False)]

,text,starts100ns,starts_sec


In [13]:
df[df.text.str.contains('seizure',case=False)] # find the seizure

,text,starts100ns,starts_sec


In [14]:
df[df.text.str.contains('spasm',case=False)] # find the seizure

,text,starts100ns,starts_sec
54,SPASM?,18367010000,1836.701
55,SPASM?,18710300000,1871.030
56,SPASM,19014460000,1901.446
57,SPASM,19388010000,1938.801


In [15]:
list(annot.items())

[('durations_char16',
  <HDF5 dataset "durations_char16": shape (66,), type "|S16">),
 ('starts_100ns', <HDF5 dataset "starts_100ns": shape (66,), type "<i8">),
 ('texts', <HDF5 dataset "texts": shape (66,), type "|O">)]

[//]: # ('cell_type':'markdown', "metadata":{'collapsed': True, 'slideshow': {'slide_type': 'skip'}})

In [16]:
# search identified spasms at 1836, 1871, 1901, 1939
fig=stacklineplot.show_epoch_centered(signals, 1836,
                        epoch_width_sec=15,
                        chstart=0, chstop=19, fs=rec.attrs['sample_frequency'],
                        ylabels=electrode_labels, yscale=3.0)
bh = show(fig, notebook_handle=True)



In [17]:
inr = stacklineplot.IpyStackplot(signals, 15, ylabels=electrode_labels, fs=rec.attrs['sample_frequency'])

In [18]:
inr.plot()
buttonf = ipywidgets.Button(description="go forward 10s")
buttonback = ipywidgets.Button(description="go backward 10s")
goto = ipywidgets.BoundedFloatText(value=0.0, min=0, max=inr.num_samples/inr.fs, step=1, description='goto time(sec)')
def go_forward(b):
    inr.loc_sec += 10
    inr.update()
    
    
def go_backward(b):
    inr.loc_sec -= 10
    inr.update()
    
def go_to_handler(change):
    # print("change:", change)
    if change['name'] == 'value':
        inr.loc_sec = change['new']
        inr.update()
    
    
buttonf.on_click(go_forward)
buttonback.on_click(go_backward)
goto.observe(handler=go_to_handler) # try using traitlet event to do callback on change

In [19]:

inr.show()
ipywidgets.HBox([buttonback,buttonf])


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [20]:
inr.data_source.data

{'xs': [array([  0.00000000e+00,   5.00000000e-03,   1.00000000e-02, ...,
           1.49850000e+01,   1.49900000e+01,   1.49950000e+01]),
  array([  0.00000000e+00,   5.00000000e-03,   1.00000000e-02, ...,
           1.49850000e+01,   1.49900000e+01,   1.49950000e+01]),
  array([  0.00000000e+00,   5.00000000e-03,   1.00000000e-02, ...,
           1.49850000e+01,   1.49900000e+01,   1.49950000e+01]),
  array([  0.00000000e+00,   5.00000000e-03,   1.00000000e-02, ...,
           1.49850000e+01,   1.49900000e+01,   1.49950000e+01]),
  array([  0.00000000e+00,   5.00000000e-03,   1.00000000e-02, ...,
           1.49850000e+01,   1.49900000e+01,   1.49950000e+01]),
  array([  0.00000000e+00,   5.00000000e-03,   1.00000000e-02, ...,
           1.49850000e+01,   1.49900000e+01,   1.49950000e+01]),
  array([  0.00000000e+00,   5.00000000e-03,   1.00000000e-02, ...,
           1.49850000e+01,   1.49900000e+01,   1.49950000e+01]),
  array([  0.00000000e+00,   5.00000000e-03,   1.00000000e-02, 

In [21]:
# this will move the window displayed in inr 
#inr.loc_sec = 1836
#inr.update()

In [22]:
inr.ys

AttributeError: 'IpyStackplot' object has no attribute 'ys'

In [ ]:
#inr.fig.multi_line(xs=inr.xs, ys=inr.ys, line_color='#8073ac')

In [ ]:
push_notebook(handle=inr.bk_handle)

In [ ]:
# f2=bplt.figure()
# f2.multi_line(xs=inr.xs, ys=inr.ys, line_color='firebrick')
# show(f2)

In [ ]:
inr.fig.xaxis.axis_label = 'seconds'

In [ ]:
inr.loc_sec=1800

In [ ]:
push_notebook(handle=inr.bk_handle)